# **Fixed Income: Bonds with Options**

- __Part 1: Mortgages__

- __Part 2: Callable Bonds__
    - Pricing
    - Option-Adjusted Spread

- __Part 3: Option-Adjusted Duration__

- __Part 4: Option-Adjusted Convexity__

## Part 1: Mortgages

- A mortgage backed security (MBS for short) is an aggregate (or pool) of mortgage loans that are either categorized as residential (RMBS) or commercial (CMBS) loans. The MBS market is further divided based on Agency assurance, where Agency MBS are backed by a government agency and Non-Agency MBS are those securities without a government stamp of approval.

- A mortgage has two _mandatory_ cash flow components:
    - Component One: scheduled interest payment
    - Component Two: scheduled principal payment (these are amortizing securities)

- A mortgage has one _non-mandatory_ cash flow component:
    - Component Three: prepayment of principal 

- Estimating Cash Flows
    - Accurately estimating the cash flows of an MBS involves modeling out the expected prepayment rate for a given horizon. This is difficult as the degree of prepayment is variable, determined by economic and psychological factors.

    - Prepayment
        - Primarily determined by refinancing, turnover, seasoning, and default
        - Measured by the conditional prepayment rate (CPR), which is an annualized percentage of the total principal that will be prepaid voluntarily/involuntarily
        - Measured by the single-month mortality rate (SMM), which is the monthly percentage of the total principal that will be prepaid voluntarily/involuntarily


### Pricing

- For a mortgage the periodic cash flows are path dependent, meaning the cash flows received in the current period are determined by the cash flows received in prevous periods. For this reason, Monte Carlo simulations are commonly used to value MBS

## Part 2: Callable Bonds

- A callable bond is a bond that has an underlying option granting the right to the issuer to buy back the bond

- For instance, a bond may have a call feature that grants the issuer the right to buy the bond back at $1100. This price will occur when interest rates decrease

- Why the call feature?
    - If a company issues bonds at 10%, but interest rates decline to 6% the company is locked in at 10% without the call feature. With the call feature they are able to buy the bonds back from the bond holder and refinance at the lower prevailing market rate

    - Thus, a callable bond can be viewed as the issuer buying a call option from the bondholder. So the issuer is long interest rate volatility and the holder is short interest rate volatility (as the option will be a function of interest rate volatility)

- Facts
    - Callable bonds have negative convexity when the market yield is lower than the call yield

    - Callable bonds have lower prices (higher yield) than similar non-callable bonds

### Pricing

- For a callable bond the pricing is dependent on the interest rate path, in other words the probability the option is exercised is based on the level of interest rates. Thus, the price of a callable bond is a function of interest rate volatility

- A binomial tree can be constructed, where each node is a future interest rate, and each branch is a path, to model future price paths under different interest rate scenarios

- The difference in price (or yield) between a callable bond and a similar option-free bond represents the value of the call option

In [41]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [42]:
class BinomialRateTree:
    def __init__(self):
        pass
    
    def _pv(self, cf: float, r: float, dt: float) -> float:
        """Calculate present value of a cash flow"""
        
        return cf / (1 + r)**dt
    
    def px_matrix(self,
                  par: float = 1000,
                  coupon_rate: float = 0.05,
                  maturity: int = 1,
                  frequency: int = 2,
                  rate_vol: float = 0.10,
                  r_0: float = 0.05,
                  call_px: float = None,
                  call_time: float = None
                  ) -> np.ndarray:
        """
        Price a bond using a binomial interest rate tree.
        
        Args:
            par: the maturity value of the bond
            coupon_rate: the annual coupon rate
            maturity: the years until maturity
            frequency: the periodic frequency of coupon payments
            rate_vol: the annual volatility of the interest rate
            r_0: the initial one-period forward rate
            call_px: the price at which the bond will be called
            call_time: the time (in years) until the bond can be called
        """
        
        n_periods = maturity * frequency
        dt = 1 / frequency
        
        n_cols = n_periods
        n_rows = 2 * n_periods - 1
        
        M = np.zeros((n_rows, n_cols), dtype=float)
        
        vol_step = rate_vol * np.sqrt(dt)
        i_mid = n_rows // 2
        
        # Build tree backwards from maturity to present
        count = 0
        for j in range(n_cols - 1, -1, -1):
            
            for i in range(count, n_rows - count, 2):
                
                r = r_0 * np.exp(vol_step * (i_mid - i))
                
                # At maturity: bond pays par + final coupon
                if j == n_cols - 1:
                    cf = par + par * coupon_rate / frequency
                    node_px = self._pv(cf, r, dt)
                    M[i, j] = node_px
                
                # At intermediate periods: discount expected value from next period
                else:
                    avg_cf = (M[i+1, j+1] + M[i-1, j+1]) / 2
                    
                    if (j + 1) % (frequency / frequency) == 0:
                        coupon = par * coupon_rate / frequency
                        node_px = self._pv(avg_cf + coupon, r, dt)
                    else:
                        node_px = self._pv(avg_cf, r, dt)
                    
                    if call_px is not None and call_time is not None:
                        call_period = int(call_time * frequency)
                        if j >= call_period:
                            node_px = min(node_px, call_px)
                    
                    M[i, j] = node_px
            
            count += 1
        
        return M
    
    def px(self,
           par: float = 1000,
           coupon_rate: float = 0.05,
           maturity: int = 1,
           frequency: int = 2,
           rate_vol: float = 0.10,
           r_0: float = 0.05,
           call_px: float = None,
           call_time: float = None
           ) -> float:
        """
        Calculate the price of a bond.
        
        Returns the present value at the root node of the tree.
        """
        M = self.px_matrix(par, coupon_rate, maturity, frequency, 
                          rate_vol, r_0, call_px, call_time)
        n_periods = maturity * frequency
        i_mid = (2 * n_periods - 1) // 2
        
        return M[i_mid, 0]

In [43]:
# Binomial Tree for pricing

tree = BinomialRateTree()

# Bond and rate parameters
par = 1000
coupon_rate = 0.07
maturity = 3
frequency = 2
rate_vol = 0.10
forward_rate = 0.065
call_px = 1010
call_time = 1

# Simple rate tree
np.round(tree.px_matrix(
    par, coupon_rate, maturity, frequency, 
    rate_vol, forward_rate, call_px, call_time), 2)

array([[   0.  ,    0.  ,    0.  ,    0.  ,    0.  ,  990.18],
       [   0.  ,    0.  ,    0.  ,    0.  ,  986.32,    0.  ],
       [   0.  ,    0.  ,    0.  ,  987.54,    0.  ,  995.76],
       [   0.  ,    0.  ,  993.15,    0.  ,  996.59,    0.  ],
       [   0.  , 1001.54,    0.  , 1001.78,    0.  , 1000.68],
       [1010.48,    0.  , 1008.63,    0.  , 1005.68,    0.  ],
       [   0.  , 1014.06,    0.  , 1010.  ,    0.  , 1005.01],
       [   0.  ,    0.  , 1010.  ,    0.  , 1010.  ,    0.  ],
       [   0.  ,    0.  ,    0.  , 1010.  ,    0.  , 1008.82],
       [   0.  ,    0.  ,    0.  ,    0.  , 1010.  ,    0.  ],
       [   0.  ,    0.  ,    0.  ,    0.  ,    0.  , 1012.16]])

In [44]:
# Get price

print(f"${np.round(tree.px(par=1000, coupon_rate=0.04, maturity=10, frequency=2, rate_vol=0.10, r_0=0.06, call_px=1050, call_time=2), 2)}")

$852.11


In [45]:
# Price Paths of Callable vs Vanilla Bonds

rates = np.arange(0.001, 0.10, 0.001)

callable_px_ls = []
vanilla_px_ls = []

for r in rates:
    rate_vol = 0.20
    par = 1000
    coupon_rate = 0.03
    maturity = 10
    tree = BinomialRateTree()

    callable_px_ls.append(tree.px(par=par, coupon_rate=coupon_rate, maturity=maturity, rate_vol=rate_vol, r_0=r, call_px=1050, call_time=1))
    vanilla_px_ls.append(tree.px(par=par, coupon_rate=coupon_rate, rate_vol=rate_vol, r_0=r, maturity=maturity))

In [46]:
# Plot Price Paths of Callable vs Vanilla Bonds

callable_df = pd.DataFrame({'rate': rates*100, 
                            'price': callable_px_ls})
vanilla_df = pd.DataFrame({'rate': rates*100,
                           'price': vanilla_px_ls})

fig = go.Figure()

fig.add_trace(go.Scatter(x=callable_df['rate'], 
                         y=callable_df['price'],
                         mode='lines',
                         name='Callable Bond'))

fig.add_trace(go.Scatter(x=vanilla_df['rate'], 
                         y=vanilla_df['price'],
                         mode='lines',
                         name='Vanilla Bond'))

fig.update_layout(
    title={
        'text': '<b>Price Paths of Callable vs Vanilla Bonds</b>',
        'x': 0.5,
        'xanchor': 'center', 
        'font': {
            'size': 24
        }
    },
    xaxis_title='Interest Rate (%)',
    yaxis_title='Bond Price ($)'
)

fig.show()

### Option-Adjusted Spread (OAS)

- Why do we use yield spread?

- 'Classic' Spread:
$$\text{yield spread} = \text{bond yield} - \text{appropriate risk-free rate}$$

- Option-adjusted spread is similar to the 'classic' yield spread; it just accounts for the underlying option in the bond

- Why is OAS useful?
    - If you are comparing two bonds, one bond that has an option and one bond without an option, you cannot simply compare the z-spread, as the bond with the option will normally trade at a higher z-spread, making it appear more attractive. You must adjust the spread for the risk of the option. When we use OAS, we are looking at the bond's spread _without_ the option. In other words, OAS gives us the option-free portion of the bond's yield spread

- OAS is calculated against a benchmark rate
    - If the benchmark rate is the Treasury spot curve, then OAS does not reflect credit risk
    - If the benchmark rate is the issuer's spot curve, then OAS reflects credit risk

- Facts:
    - Lower interest rates --> value of call option increases --> OAS decreases
    - Higher interest rates --> value of call option decreases --> OAS increases
    - Higher interest rate volatility --> value of call option increases --> lower OAS
    - Lower interest rate volatility --> value of call option decreases --> higher OAS

- The option's value in terms of spread:
$$\text{option value (bps)} = \text{static spread} - \text{OAS}$$

In [47]:
# Calculating OAS

def build_rate_tree(r0, u, d, periods):
    tree = {}
    for t in range(periods + 1):
        for i in range(t + 1):
            tree[(t, i)] = r0 * (u ** (t - i)) * (d ** i)
    return tree

def calculate_oas(face_value, coupon_rate, years, r0, u, d, call_price, call_start, oas_guess=0.0):
    periods = years * 2
    coupon = face_value * coupon_rate / 2
    
    rate_tree = build_rate_tree(r0, u, d, periods)
    
    oas = oas_guess
    tolerance = 0.0001
    max_iter = 50
    
    for _ in range(max_iter):
        value_tree = {}
        
        for i in range(periods + 1):
            value_tree[(periods, i)] = face_value + coupon
        
        for t in range(periods - 1, -1, -1):
            for i in range(t + 1):
                rate = rate_tree[(t, i)] + oas
                
                up_value = value_tree[(t + 1, i)]
                down_value = value_tree[(t + 1, i + 1)]
                expected_value = (0.5 * up_value + 0.5 * down_value) / (1 + rate / 2)
                
                expected_value += coupon
                
                if t >= call_start:
                    expected_value = min(expected_value, call_price)
                
                value_tree[(t, i)] = expected_value
        
        calculated_price = value_tree[(0, 0)]
        
        if abs(calculated_price - face_value) < tolerance:
            return oas * 10000
        
        if calculated_price > face_value:
            oas += 0.0001
        else:
            oas -= 0.0001
    
    return oas * 10000

In [48]:
# Calculate OAS for a callable bond

face = 100
coupon = 0.05
maturity = 5
    
r0 = 0.04
u = 1.15
d = 0.90
    
call_price = 1020
call_start = 4
    
oas_bp = calculate_oas(face, coupon, maturity, r0, u, d, call_price, call_start)
    
print(f"OAS: {oas_bp:.2f} bps")

OAS: 50.00 bps


# Part 3: Option-Adjusted Duration (OAD)

- Option-adjusted duration, or effective duration is similar to modified duration. However, unlike modified duration, effective duration considers the change in cash flow for a change in interest rates 

- Effective duration is used for bonds with options

- Effective duration can be quite different than modified duration for a bond with an option

- Effective duration is the same as modified duration for a bond without an option

$$\text{Effective Duration} = \frac{P_d - P_u}{2 \times P_0 \times \Delta y}$$

Where:
- $P_d$ is the price of the bond at _lower_ yield calculated using a method that considers the option
- $P_u$ is the price of the bond at _higher_ yield calculated using a method that considers the option
- $P_0$ is the price of the bond with _no-change_ to yield calculated using a method that considers the option
- $\Delta y$ is the change in yield (yield shock)

# Part 4: Option-Adjusted Convexity (OAC)

- Option-adjusted convexity, or effective convexity is similar to convexity. However, unlike convexity, effective convexity considers the change in cash flow for a change in interest rates 

- Effective convexity is used for bonds with options

- Effective convexity can be quite different than modified duration for a bond with an option

- Effective convexity is the same as modified duration for a bond without an option

$$\text{Effective Convexity} = \frac{P_u + P_d - 2(P_0)}{(P_0) (\Delta y)^2}$$

Where:
- $P_d$ is the price of the bond at _lower_ yield calculated using a method that considers the option
- $P_u$ is the price of the bond at _higher_ yield calculated using a method that considers the option
- $P_0$ is the price of the bond with _no-change_ to yield calculated using a method that considers the option
- $\Delta y$ is the change in yield (yield shock)

### References

1. **Fabozzi, F. J., & Fabozzi, F. A.** (2021). *Bond Markets, Analysis, and Strategies* (10th ed.). MIT Press.

2. **Hull, J. C.** (2018). *Options, Futures, and Other Derivatives* (10th ed.). Pearson.